In [95]:
import os
import glob

import matplotlib.pyplot as plt

import pandas as pd
import geopandas as gpd

from keplergl import KeplerGl

# Tiền xử lý dữ liệu

Một tập dữ liệu với tất cả thông tin do chính phủ cung cấp về covid19 (tên gọi, giới tính, ngày, số người hiện đang nằm viện, số người hiện đang được hồi sức hoặc chăm sóc nguy kịch, tổng số bệnh nhân đã trở về nhà, tổng số người chết tại bệnh viện.

* **sexe : Giới tính**
* **jour: Ngày** 
* **hosp: Số người hiện đang nằm viện** 
* **rea: Số người đang hồi sức hoặc được chăm sóc tích cực** 
* **rad: Số bệnh nhân đã trở về nhà** 
* **dc: Số bệnh nhân đã mất tại viện** 

In [96]:
covid_gouv = 'data/data-hospital-covid19-2020-04-13-19h00.csv'
df_gouv    = pd.read_csv(covid_gouv, delimiter = ';')

df_gouv.head()

,dep,sexe,jour,hosp,rea,rad,dc
0,01,0,2020-03-18,2,0,1,0
1,01,1,2020-03-18,1,0,1,0
2,01,2,2020-03-18,1,0,0,0
3,02,0,2020-03-18,41,10,18,11
4,02,1,2020-03-18,19,4,11,6


Lọc lấy dữ liệu lớn nhất theo từng ngày của từng khu vực (code) vì lý do dữ liệu của từng khu vực được cập nhật nhiều lần trong ngày nên chỉ cần lấy số liệu lớn nhất

In [97]:
df_day = df_gouv['jour'].drop_duplicates() #Loại bỏ dữ liệu trùng
max = df_day.max() #Lấy ngày lớn nhất (ngày cuối cùng của bộ dữ liệu)

df_gouv['code'] = df_gouv['dep'] #Gán code bằng dep để sau này sẽ dùng code nối các bảng

temp_df = dict() #Tạo 1 dict tạm 
for i in df_day :
    temp_df[i] = df_gouv.loc[ df_gouv['jour'] == i] #Tổng hợp dữ liệu lại theo từng ngày
dict_df = dict()
for key, df in temp_df.items() :
    dict_df[key] = df.groupby('code').max().drop(['sexe','dep'], axis=1) #Nhóm theo code và lấy dữ liệu lớn nhất 

dict_df[max]

,jour,hosp,rea,rad,dc
code,,,,,
01,2020-04-13,120,30,139,44
02,2020-04-13,286,44,288,147
03,2020-04-13,58,20,82,13
04,2020-04-13,33,4,63,5
05,2020-04-13,49,13,70,3
...,...,...,...,...,...
971,2020-04-13,41,18,51,10
972,2020-04-13,39,16,52,6
973,2020-04-13,15,1,13,0


### Demo Bộ Dữ Liệu Địa Lý Và Cư Dân

#### A dataset with useful demographic statistics. Source : https://www.regions-et-departements.fr/departements-francais

In [98]:
demographic = 'data/demographie.csv'
df_de       = pd.read_csv(demographic, delimiter = ',')
df_de       = df_de.set_index('code') #Đặt index là code
df_de.head()

,nom,Superficie,Population,Densite
code,,,,
01,Ain,5762,631877,109.7
02,Aisne,7369,538659,73.1
03,Allier,7340,341613,46.5
04,Alpes-de-Haute-Provence,6925,161799,23.4
05,Hautes-Alpes,5549,140916,25.4


## Dữ liệu phân nhóm theo độ tuổi, giới tính, dân cư của từng vùng
Source : https://www.insee.fr/fr/statistiques/2012692#graphique-TCRD_021_tab1_departements

In [99]:
demographic_1  = 'data/TCRD_021.csv'
df_de1         = pd.read_csv(demographic_1, delimiter = ',')
df_de1['code'] = df_de1['Unnamed: 0'] #Lấy cột Unnamed: 0 làm code
df_de1         = df_de1.set_index('code').drop(['Unnamed: 0','Unnamed: 1'], axis =1 ) #Set cột code làm index loại bỏ các cột Unnamed
df_de1         = df_de1.dropna() #Xóa giá trị bị thiếu

df_de1.head()

,Ensemble,Part des femmes (en %),Part des hommes (en %),Part des 0 à 24 ans (en %),Part des 25 à 59 ans (en %),Part des 60 ans ou plus (en %),dont part des 75 ans ou plus (en %)
code,,,,,,,
01,656955.0,50.8,49.2,30.2,45.6,24.2,8.3
02,526050.0,51.1,48.9,29.5,42.6,27.9,9.3
03,331315.0,52.2,47.8,24.3,40.2,35.6,13.8
04,165197.0,51.2,48.8,24.5,41.2,34.3,12.7
05,141756.0,51.2,48.8,25.2,42.4,32.4,11.9


## Geospatial data

### A geospatial dataset with all the departement in France and their geometry (geojson format) Thanks to https://github.com/gregoiredavid
### Dữ liệu geo data với các tọa độ theo từng vùng ở Pháp 

In [100]:
geospatial_data  = 'data/departements-avec-outre-mer.geojson'
f = gpd.read_file(geospatial_data) #đọc file dữ liệu địa lý
f = f.set_index('code') #set index là code
f = f.sort_index(axis=0) #sắp xếp lại index

f.head()

,nom,geometry
code,,
01,Ain,"POLYGON ((4.78021 46.17668, 4.78024 46.18905, ..."
02,Aisne,"POLYGON ((3.17296 50.01131, 3.17382 50.01186, ..."
03,Allier,"POLYGON ((3.03207 46.79491, 3.03424 46.79080, ..."
04,Alpes-de-Haute-Provence,"POLYGON ((5.67604 44.19143, 5.67817 44.19051, ..."
05,Hautes-Alpes,"POLYGON ((6.26057 45.12685, 6.26417 45.12641, ..."


# DataFrame Coronavirus

Bước đầu tiên là quyết định chỉ mục nào sẽ được sử dụng để nối tất cả dữ liệu.

Mục đích là truy xuất dữ liệu không gian địa lý, vì vậy chúng ta cần nối dữ liệu df_geo và covid19 để có cột hình học.


Tôi quyết định chọn số code. Thật vậy, tất cả dữ liệu này đều chứa số hiệu. Tuy nhiên, df_gouv là một chuỗi thời gian nên để có thể sử dụng số departement, tôi chia dữ liệu trong nhiều DataFrame. Mỗi DataFrame là một ngày cụ thể.

* Mật độ dân số bị ảnh hưởng (tử vong/mật độ dân cư)
* người trên 75 tuổi (%) (Cỏ thể được thay thế bằng các nhóm tuổi khác)
* Số người bị ảnh hưởng (tử vong/số dân số)
* Tỷ lệ tử vong/tổng số còn sống(số còn đang nằm viện + số đã hồi phục)
* Tỷ lệ nhập viện (Số người nằm viện/tổng dân số)

=> Tính được danh sách covid theo từng ngày và từng vùng

In [101]:
dict_coronavirus = dict()

for i in df_day :
    dict_coronavirus[i] = pd.concat([f,dict_df[i]], axis = 1) #Kết hợp dữ liệu covid và dữ liệu địa lý
    dict_coronavirus[i]['dc/densite']    = dict_coronavirus[i]['dc'] / df_de['Densite'] #Cột dữ liệu tỉ lệ Số bệnh nhân đã mất tại bệnh viện/Mật độ => mật độ dân số bị ảnh hưởng
    dict_coronavirus[i]['people older than 75 year (%)'] = df_de1['dont part des 75 ans ou plus (en %)']
    dict_coronavirus[i]['dc/population'] = dict_coronavirus[i]['dc'] / df_de['Population']
    dict_coronavirus[i]['dc/cumul hospitalise'] = dict_coronavirus[i]['dc'] / (dict_coronavirus[i]['hosp']+dict_coronavirus[i]['rad'])
    dict_coronavirus[i]['cumul hopitalise / population']     = (dict_coronavirus[i]['hosp']+dict_coronavirus[i]['rad']) / df_de['Population']

dict_coronavirus[max].head()

,nom,geometry,jour,hosp,rea,rad,dc,dc/densite,people older than 75 year (%),dc/population,dc/cumul hospitalise,cumul hopitalise / population
code,,,,,,,,,,,,
01,Ain,"POLYGON ((4.78021 46.17668, 4.78024 46.18905, ...",2020-04-13,120,30,139,44,0.401094,8.3,0.000070,0.169884,0.000410
02,Aisne,"POLYGON ((3.17296 50.01131, 3.17382 50.01186, ...",2020-04-13,286,44,288,147,2.010944,9.3,0.000273,0.256098,0.001066
03,Allier,"POLYGON ((3.03207 46.79491, 3.03424 46.79080, ...",2020-04-13,58,20,82,13,0.279570,13.8,0.000038,0.092857,0.000410
04,Alpes-de-Haute-Provence,"POLYGON ((5.67604 44.19143, 5.67817 44.19051, ...",2020-04-13,33,4,63,5,0.213675,12.7,0.000031,0.052083,0.000593
05,Hautes-Alpes,"POLYGON ((6.26057 45.12685, 6.26417 45.12641, ...",2020-04-13,49,13,70,3,0.118110,11.9,0.000021,0.025210,0.000844


# Thống Kê
**Đặt ngưỡng 400 tổng số người đang nằm viện và đã hồi phục phải lớn hơn ngưỡng**

In [102]:
df_stats = dict_coronavirus[max]
threshold = 400

df_stats['dead/alive'] = df_stats.loc[(df_stats.hosp + df_stats.rad) > threshold]['dc/cumul hospitalise']
df_stats['dead/alive'] = df_stats['dead/alive'].fillna(0) #Fill 0 vào những dòng k có giá trị

df_stats['dead/densite'] = df_stats.loc[(df_stats.hosp + df_stats.rad) > threshold]['dc/densite']
df_stats['dead/densite'] = df_stats['dead/densite'].fillna(0)

df_stats['dead/population'] = df_stats.loc[(df_stats.hosp + df_stats.rad) > threshold]['dc/population']
df_stats['dead/population'] = df_stats['dead/population'].fillna(0)

df_stats = df_stats.drop(['cumul hopitalise / population','dc/cumul hospitalise','dc/population','dc/densite'], axis=1)
df_stats.head()

,nom,geometry,jour,hosp,rea,rad,dc,people older than 75 year (%),dead/alive,dead/densite,dead/population
code,,,,,,,,,,,
01,Ain,"POLYGON ((4.78021 46.17668, 4.78024 46.18905, ...",2020-04-13,120,30,139,44,8.3,0.000000,0.000000,0.000000
02,Aisne,"POLYGON ((3.17296 50.01131, 3.17382 50.01186, ...",2020-04-13,286,44,288,147,9.3,0.256098,2.010944,0.000273
03,Allier,"POLYGON ((3.03207 46.79491, 3.03424 46.79080, ...",2020-04-13,58,20,82,13,13.8,0.000000,0.000000,0.000000
04,Alpes-de-Haute-Provence,"POLYGON ((5.67604 44.19143, 5.67817 44.19051, ...",2020-04-13,33,4,63,5,12.7,0.000000,0.000000,0.000000
05,Hautes-Alpes,"POLYGON ((6.26057 45.12685, 6.26417 45.12641, ...",2020-04-13,49,13,70,3,11.9,0.000000,0.000000,0.000000


# Chọn ngày

In [103]:
print('Choose Date :')

print(df_day.min())
print(df_day.max())

Choose Date :
2020-03-18
2020-04-13


In [104]:
day = '2020-04-13'

# Phân tích lấy dữ liệu theo ngày

In [105]:
daily_coronavirus = dict_coronavirus[day]

daily_coronavirus.head()

,nom,geometry,jour,hosp,rea,rad,dc,dc/densite,people older than 75 year (%),dc/population,dc/cumul hospitalise,cumul hopitalise / population,dead/alive,dead/densite,dead/population
code,,,,,,,,,,,,,,,
01,Ain,"POLYGON ((4.78021 46.17668, 4.78024 46.18905, ...",2020-04-13,120,30,139,44,0.401094,8.3,0.000070,0.169884,0.000410,0.000000,0.000000,0.000000
02,Aisne,"POLYGON ((3.17296 50.01131, 3.17382 50.01186, ...",2020-04-13,286,44,288,147,2.010944,9.3,0.000273,0.256098,0.001066,0.256098,2.010944,0.000273
03,Allier,"POLYGON ((3.03207 46.79491, 3.03424 46.79080, ...",2020-04-13,58,20,82,13,0.279570,13.8,0.000038,0.092857,0.000410,0.000000,0.000000,0.000000
04,Alpes-de-Haute-Provence,"POLYGON ((5.67604 44.19143, 5.67817 44.19051, ...",2020-04-13,33,4,63,5,0.213675,12.7,0.000031,0.052083,0.000593,0.000000,0.000000,0.000000
05,Hautes-Alpes,"POLYGON ((6.26057 45.12685, 6.26417 45.12641, ...",2020-04-13,49,13,70,3,0.118110,11.9,0.000021,0.025210,0.000844,0.000000,0.000000,0.000000


## Đánh giá mức độ nguy cơ đối với nhóm đối tượng theo từng vùng

### Demographic Data

#### Đối với những người già >75 tuổi

Nếu mật độ người già < 9 => mức độ 1
Nếu mật độ người già > 9 và < 10.6 => mức độ 2
Nếu mật độ người già < 10.6 và < 12.5 => mức độ 3
Nếu mật độ người già > 12.5 => mức độ 4

In [106]:
cat_0 = 0
#Rủi ro = 1
cat_1 = 9.000000 
#Rủi ro = 2
cat_2 = 10.600000
#Rủi ro = 3
cat_3 = 12.500000
#Rủi ro = 4  

df_temp = df_de1.drop(['Ensemble','Part des femmes (en %)', 'Part des hommes (en %)','Part des 0 à 24 ans (en %)', 'Part des 25 à 59 ans (en %)', 'Part des 60 ans ou plus (en %)'], axis = 1)
x = df_temp['dont part des 75 ans ou plus (en %)']
df_temp['Nguy cơ tuổi già'] = df_temp['dont part des 75 ans ou plus (en %)'].apply(lambda x : 1 if x < cat_1 else (2 if x > cat_1 and x < cat_2 else (3 if x > cat_2 and x < cat_3 else 4)))
df_temp = df_temp.drop(['dont part des 75 ans ou plus (en %)'], axis = 1)

df_temp

,Nguy cơ tuổi già
code,
01,1
02,2
03,4
04,4
05,3
...,...
971,2
972,4
973,1


## Đánh giá mức độ lây nhiễm cùng với rủi ro theo từng vùng đã được tính trước đó

### Đánh giá dựa vào giá trị số người lây nhiễm / tổng dân số khu vực

In [107]:
cat_0 = 0
#Risque = 1
cat_1 = 0.000284
#Risque = 2
cat_2 = 0.000494
#Risque =  3
cat_3 = 0.000898
#Risque = 3

x1 = dict_coronavirus[max]['cumul hopitalise / population']
df_temp['Rủi ro lây nhiễm'] = dict_coronavirus[max]['cumul hopitalise / population'].apply(lambda x : 4 if x < cat_1 else (3 if x > cat_1 and x < cat_2 else (2 if x > cat_2 and x < cat_3 else 1)))

df_temp

,Nguy cơ tuổi già,Rủi ro lây nhiễm
code,,
01,1,3
02,2,1
03,4,3
04,4,2
05,3,2
...,...,...
971,2,4
972,4,4
973,1,4


# Đánh giá mức độ nguy hiểm của từng khu vực bằng cách lấy tích rủi ro lây nhiễm với nguy cơ tuổi già

In [108]:
df_temp['Điểm'] = df_temp['Nguy cơ tuổi già'] * df_temp['Rủi ro lây nhiễm']
    
#df_temp.head()
df_temp

,Nguy cơ tuổi già,Rủi ro lây nhiễm,Điểm
code,,,
01,1,3,3
02,2,1,2
03,4,3,12
04,4,2,8
05,3,2,6
...,...,...,...
971,2,4,8
972,4,4,16
973,1,4,4


# Gộp dữ liệu covid với dữ liệu dự đoán mức độ nguy hiểm

In [109]:
df_risque = pd.concat([f,df_temp], axis = 1)

df_risque.head()

,nom,geometry,Nguy cơ tuổi già,Rủi ro lây nhiễm,Điểm
01,Ain,"POLYGON ((4.78021 46.17668, 4.78024 46.18905, ...",1,3,3
02,Aisne,"POLYGON ((3.17296 50.01131, 3.17382 50.01186, ...",2,1,2
03,Allier,"POLYGON ((3.03207 46.79491, 3.03424 46.79080, ...",4,3,12
04,Alpes-de-Haute-Provence,"POLYGON ((5.67604 44.19143, 5.67817 44.19051, ...",4,2,8
05,Hautes-Alpes,"POLYGON ((6.26057 45.12685, 6.26417 45.12641, ...",3,2,6


# Xử dụng thư viện KeplerGl để vẽ bản đồ

In [110]:
map_stats = KeplerGl(height=650)
map_stats.add_data(df_stats, name ='coronavirus')
map_stats

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'coronavirus': {'index': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12…

# Tạo và lưu trữ file config từ bản đồ đã được khởi tạo

In [111]:
config_stats = map_stats.config
with open('config_stats.py', 'w') as f:
    f.write('config = {}'.format(config_stats))

# Lưu bản đồ vào html

In [112]:
# this will save current map
map_stats.save_to_html(file_name='map_stats.html')

Map saved to map_stats.html!


In [113]:
map_coronavirus_2D = KeplerGl(height=800)
map_coronavirus_2D.add_data(dict_coronavirus['2020-04-13'], name ='coronavirus')
map_coronavirus_2D

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'coronavirus': {'index': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12…

In [114]:
# # this will save current map
map_coronavirus_2D.save_to_html(file_name='map_coronavirus_2D.html')

Map saved to map_coronavirus_2D.html!


In [115]:
map_risk = KeplerGl(height=500)
map_risk.add_data(df_risque, name = 'risk')
map_risk

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'risk': {'index': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13'…

In [116]:
map_risk.save_to_html(file_name='map_risk_2D.html')

Map saved to map_risk_2D.html!
